# Deep Mailing - XGBoost Model - Calculado Correlacoes

O objetivo desse notebook é demonstrar a utilizacao do XGBoost para a criacao de arvores de decisão para a predição de CUPS em mailings.

Em primeiro lugar, definimos os imports que iremos usar...

In [1]:
import xgboost
import numpy as np
import os
import sys
import logging
import gc
import pickle as pickle
import pandas as pd
import dateutil.parser as parser
import os.path
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score
from scipy.stats import pearsonr
from datetime import datetime
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

/home/ubuntu/git/DeepMailing/env/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Abaixo definimos os diretorios e nomes dos arquivos intermediarios.

In [15]:
log_location = "../logs/"
arquivo_df_pickled_norm = "../intermediate/df.norm.pickle"
arquivo_df_pickled_norm_corr = "../intermediate/df.norm.corr.pickle"
arquivo_df_pickled_norm_train = "../intermediate/df.norm.train.pickle"
arquivo_df_pickled_norm_test = "../intermediate/df.norm.test.pickle"
arquivo_df_pickled_norm_train_x = "../intermediate/df.norm.train.x.pickle.npy"
arquivo_df_pickled_norm_train_y = "../intermediate/df.norm.train.y.pickle.npy"
arquivo_df_pickled_norm_test_x = "../intermediate/df.norm.test.x.pickle.npy"
arquivo_df_pickled_norm_test_y = "../intermediate/df.norm.test.y.pickle.npy"

Redefinimos o logger que iremos usar

In [3]:
logger = logging.getLogger()
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'xgboost.log.' + datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

Criamos uma função para imprimir tanto no log quanto no notebook...

In [4]:
def print_log(msg):
    logging.debug(msg)
    print(msg)    

Carregamos para a memoria o arquivo normalizado e pickled que foi gerado no notebook de "Preparação de Dados".

In [5]:
print_log("Carregando Pickling normalizado:{}".format(arquivo_df_pickled_norm))    
chamadas = pd.read_pickle(arquivo_df_pickled_norm)

Carregando Pickling normalizado:../intermediate/df.norm.pickle


Verificamos as dimensões do dataframe carregado.... E imprimimos uma amostra do dado que precisamos com apenas as colunas relevantes... Como podemos perceber o nosso modelo considera apenas colunas com valores booleanos (0 ou 1)

In [6]:
chamadas.head(10)

,id,PROPENSAO,NUMERO,DATA_MAILING,NORM_CARTEIRA_W02,NORM_CARTEIRA_W01,NORM_CARTEIRA_A02,NORM_CARTEIRA_A01,NORM_CARTEIRA_A03,NORM_SEGMENTO_FA,...,NORM_DDD_77,NORM_DDD_49,NORM_DDD_74,NORM_DDD_62,NORM_DDD_37,NORM_DDD_12,NORM_DDD_89,NORM_TENTATIVAS,NORM_LIGACOES,NORM_CUP
0,1808947,NaN,-996934974.0,2017-11-28,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1808948,NaN,-996038652.0,2017-11-28,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
2,1808949,NaN,-996581839.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1808950,ALTA PROPENSÃƒO DE PAGAMENTO PARCELADO.,-32234178.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1808951,NaN,-32365578.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1808952,ALTA PROPENSÃƒO DE PAGAMENTO PARCELADO.,-998764576.0,2017-11-28,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
6,1808953,NaN,-38084249.0,2017-11-28,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1808954,ALTA PROPENSÃƒO DE PAGAMENTO PARCELADO.,-988019800.0,2017-11-28,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,1808955,NaN,-999254115.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1808956,NaN,-994169573.0,2017-11-28,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [7]:
print_log(chamadas.shape)
chamadas.loc[:, 'NORM_CARTEIRA_W02':'NORM_LIGACOES'].head(10)

(9586480, 106)


,NORM_CARTEIRA_W02,NORM_CARTEIRA_W01,NORM_CARTEIRA_A02,NORM_CARTEIRA_A01,NORM_CARTEIRA_A03,NORM_SEGMENTO_FA,NORM_SEGMENTO_LC,NORM_SEGMENTO_CR,NORM_SEGMENTO_FC,NORM_SEGMENTO_CC,...,NORM_DDD_41,NORM_DDD_77,NORM_DDD_49,NORM_DDD_74,NORM_DDD_62,NORM_DDD_37,NORM_DDD_12,NORM_DDD_89,NORM_TENTATIVAS,NORM_LIGACOES
0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
6,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


Para tratarmos as colunas de forma correta, precisamos eliminar os caracteres especiais das colunas, principalmente o espaco e o sinal de - para isso iremos rodar o codigo abaixo para tratar.

In [8]:
cols = chamadas.columns
cols = cols.map(lambda x: x.replace(' ', '_').replace('-','menos'))
chamadas.columns = cols

Vimos q na realidade ter LIGACOES é fundamental para ter CUP...

In [9]:
chamadas_calculo_correlacao = chamadas.loc[:, 'NORM_CARTEIRA_W02':'NORM_CUP'].query('NORM_CUP >= 0')

In [10]:
print_log("Registros com CUP:{}".format(len(chamadas_calculo_correlacao)))

Registros com CUP:9586480


In [11]:
chamadas_calculo_correlacao_cup = chamadas_calculo_correlacao.loc[:, 'NORM_CUP':'NORM_CUP'].as_matrix().ravel().astype(float)
print(chamadas_calculo_correlacao_cup)

[ 0.  0.  0. ...,  0.  0.  0.]


In [12]:
corr_campos_computados = { "coluna" : [], "corr": [] }

for coluna in chamadas_calculo_correlacao.columns.values:
    if coluna == 'NORM_CUP':
        continue
    chamadas_calculo_correlacao_coluna = chamadas_calculo_correlacao.loc[:, coluna:coluna].as_matrix().ravel().astype(float)
    corr = abs(pearsonr(chamadas_calculo_correlacao_coluna, chamadas_calculo_correlacao_cup)[0] * 100)
    corr_campos_computados['coluna'] = corr_campos_computados['coluna'] + [coluna]
    corr_campos_computados['corr'] = corr_campos_computados['corr'] + [corr]

corr = pd.DataFrame.from_dict(corr_campos_computados).sort_values('corr',ascending = False)
corr.head(20)

/home/ubuntu/git/DeepMailing/env/lib/python3.5/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


,coluna,corr
100,NORM_LIGACOES,10.540032
31,NORM_STATUS_TELEFONE_Hot,8.968208
21,NORM_STATUS_INTERNA_Hot,8.968208
19,NORM_STATUS_BUREAU_Bom,3.863116
22,NORM_STATUS_INTERNA_Indefinido,3.732327
3,NORM_CARTEIRA_A01,3.096182
28,NORM_STATUS_TELEFONE_Indefinido,2.998525
18,NORM_STATUS_BUREAU_Indefinido,1.624854
0,NORM_CARTEIRA_W02,1.586312
27,NORM_STATUS_TELEFONE_Bom,1.556349


In [14]:
colunas_finais = []
for coluna in corr.coluna:
    if coluna == 'NORM_TENTATIVAS':
        continue
    if coluna == 'NORM_LIGACOES':
        continue        
    if 'nan' in coluna:
        continue
    if 'DDD' in coluna:
        break
    colunas_finais = colunas_finais + [coluna]

colunas_finais = colunas_finais + ['NORM_TENTATIVAS', 'NORM_LIGACOES', 'NORM_CUP']
chamadas_novo = chamadas[colunas_finais]
chamadas_novo.head(10)

,NORM_STATUS_TELEFONE_Hot,NORM_STATUS_INTERNA_Hot,NORM_STATUS_BUREAU_Bom,NORM_STATUS_INTERNA_Indefinido,NORM_CARTEIRA_A01,NORM_STATUS_TELEFONE_Indefinido,NORM_STATUS_BUREAU_Indefinido,NORM_CARTEIRA_W02,NORM_STATUS_TELEFONE_Bom,NORM_ORIGEM_BUREAU,...,NORM_ORIGEM_BASE_INTERNA,NORM_CARTEIRA_A03,NORM_CARTEIRA_A02,NORM_STATUS_INTERNA_Validado,NORM_CARTEIRA_W01,NORM_STATUS_TELEFONE_Sem_Tel,NORM_STATUS_TELEFONE_Validado,NORM_TENTATIVAS,NORM_LIGACOES,NORM_CUP
0,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,1,1,0
2,0,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
6,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
7,0,0,1,0,0,0,0,0,1,1,...,0,0,0,0,1,0,0,1,0,0
8,1,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0


In [17]:
chamadas.to_pickle(arquivo_df_pickled_norm_corr)